In [1]:
import pandas as pd
import numpy as np

In [2]:
from keras import backend as K
from keras.layers import Input, Embedding, merge

Using TensorFlow backend.


In [42]:
from keras.regularizers import l2, l1
from keras.layers.core import Flatten, Dense, Dropout, Lambda
from keras.models import Sequential, Model
from keras.optimizers import SGD, RMSprop, Adam
from keras.layers import Input, Dense


In [114]:
matches = pd.read_csv('lol_matches_imputed2')# this imputation method uses avgs across dataset  missing data. , the other uses a static 1 win 2 losses

In [115]:
matches.shape

(71933, 84)

In [116]:
matches['patch_version'].value_counts()

6.23.166.2676    57118
6.22.165.9605     2575
7.13.192.6794     1934
7.12.190.9002     1183
7.10.187.9675     1107
6.24.169.4424      854
7.11.189.942       831
7.9.186.8155       777
7.5.178.6844       753
7.2.174.829        713
7.6.181.2079       672
7.8.184.113        640
7.3.175.271        604
7.7.182.1428       431
7.1.172.6393       351
7.4.176.9828       292
7.7.183.8394       257
7.1.171.6847       234
7.4.178.5050       233
6.24.168.1268      129
7.9.186.1051       126
7.2.173.3544        58
7.6.180.6903        57
6.1.0.484            4
Name: patch_version, dtype: int64

Despite my best effors, I have a larqe quantity of 6.23 matches. 

In [117]:
matches_s6 = matches[matches['patch_version']=='6.23.166.2676']
#matches_s713= matches[matches['patch_version']=='7.13.192.6794']
#matches_s712= matches[matches['patch_version']=='7.12.190.9002']

In [118]:
#matches_s6 = matches


In [119]:
matches_s6.columns

Index(['Unnamed: 0', 'patch_version', 'team1DUO_CARRY', 'team1DUO_CARRY_cl',
       'team1DUO_CARRY_cpp', 'team1DUO_CARRY_cw', 'team1DUO_CARRY_cwp',
       'team1DUO_CARRY_ol', 'team1DUO_CARRY_ow', 'team1DUO_CARRY_owp',
       'team1DUO_SUPPORT', 'team1DUO_SUPPORT_cl', 'team1DUO_SUPPORT_cpp',
       'team1DUO_SUPPORT_cw', 'team1DUO_SUPPORT_cwp', 'team1DUO_SUPPORT_ol',
       'team1DUO_SUPPORT_ow', 'team1DUO_SUPPORT_owp', 'team1JUNGLE',
       'team1JUNGLE_cl', 'team1JUNGLE_cpp', 'team1JUNGLE_cw',
       'team1JUNGLE_cwp', 'team1JUNGLE_ol', 'team1JUNGLE_ow',
       'team1JUNGLE_owp', 'team1MIDDLE', 'team1MIDDLE_cl', 'team1MIDDLE_cpp',
       'team1MIDDLE_cw', 'team1MIDDLE_cwp', 'team1MIDDLE_ol', 'team1MIDDLE_ow',
       'team1MIDDLE_owp', 'team1TOP', 'team1TOP_cl', 'team1TOP_cpp',
       'team1TOP_cw', 'team1TOP_cwp', 'team1TOP_ol', 'team1TOP_ow',
       'team1TOP_owp', 'team1_win', 'team2DUO_CARRY', 'team2DUO_CARRY_cl',
       'team2DUO_CARRY_cpp', 'team2DUO_CARRY_cw', 'team2DUO_CARRY_

In [120]:
matches_s6['team1']=matches_s6['team1_win'].astype('int')
matches_s6['team2']=matches_s6['team2_win'].astype('int')
matches_s6.head()

/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.
/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


,Unnamed: 0,patch_version,team1DUO_CARRY,team1DUO_CARRY_cl,team1DUO_CARRY_cpp,team1DUO_CARRY_cw,team1DUO_CARRY_cwp,team1DUO_CARRY_ol,team1DUO_CARRY_ow,team1DUO_CARRY_owp,...,team2TOP_cl,team2TOP_cpp,team2TOP_cw,team2TOP_cwp,team2TOP_ol,team2TOP_ow,team2TOP_owp,team2_win,team1,team2
927,1116,6.23.166.2676,81.0,2.0,0.004975,1.0,0.333333,287.0,316.0,0.524046,...,2.0,0.006369,1.0,0.333333,209.0,262.0,0.556263,True,0,1
928,1117,6.23.166.2676,202.0,2.0,0.002620,1.0,0.333333,548.0,597.0,0.521397,...,2.0,0.003030,1.0,0.333333,477.0,513.0,0.518182,True,0,1
929,1118,6.23.166.2676,119.0,2.0,0.029126,1.0,0.333333,32.0,71.0,0.689320,...,2.0,0.008065,1.0,0.333333,182.0,190.0,0.510753,False,1,0
930,1119,6.23.166.2676,81.0,2.0,0.004769,1.0,0.333333,260.0,369.0,0.586645,...,2.0,0.002611,1.0,0.333333,570.0,579.0,0.503916,False,1,0
931,1120,6.23.166.2676,202.0,2.0,0.004769,1.0,0.333333,260.0,369.0,0.586645,...,2.0,0.005068,1.0,0.333333,270.0,322.0,0.543919,False,1,0


In [121]:
laneslist = [
    'team1TOP',
    'team1MIDDLE',
    'team1JUNGLE',
    'team1DUO_CARRY',
    'team1DUO_SUPPORT',
    'team2TOP',
    'team2MIDDLE',
    'team2JUNGLE',
    'team2DUO_CARRY',
    'team2DUO_SUPPORT'
    
    
]

In [122]:
list(matches_s6.columns)

['Unnamed: 0',
 'patch_version',
 'team1DUO_CARRY',
 'team1DUO_CARRY_cl',
 'team1DUO_CARRY_cpp',
 'team1DUO_CARRY_cw',
 'team1DUO_CARRY_cwp',
 'team1DUO_CARRY_ol',
 'team1DUO_CARRY_ow',
 'team1DUO_CARRY_owp',
 'team1DUO_SUPPORT',
 'team1DUO_SUPPORT_cl',
 'team1DUO_SUPPORT_cpp',
 'team1DUO_SUPPORT_cw',
 'team1DUO_SUPPORT_cwp',
 'team1DUO_SUPPORT_ol',
 'team1DUO_SUPPORT_ow',
 'team1DUO_SUPPORT_owp',
 'team1JUNGLE',
 'team1JUNGLE_cl',
 'team1JUNGLE_cpp',
 'team1JUNGLE_cw',
 'team1JUNGLE_cwp',
 'team1JUNGLE_ol',
 'team1JUNGLE_ow',
 'team1JUNGLE_owp',
 'team1MIDDLE',
 'team1MIDDLE_cl',
 'team1MIDDLE_cpp',
 'team1MIDDLE_cw',
 'team1MIDDLE_cwp',
 'team1MIDDLE_ol',
 'team1MIDDLE_ow',
 'team1MIDDLE_owp',
 'team1TOP',
 'team1TOP_cl',
 'team1TOP_cpp',
 'team1TOP_cw',
 'team1TOP_cwp',
 'team1TOP_ol',
 'team1TOP_ow',
 'team1TOP_owp',
 'team1_win',
 'team2DUO_CARRY',
 'team2DUO_CARRY_cl',
 'team2DUO_CARRY_cpp',
 'team2DUO_CARRY_cw',
 'team2DUO_CARRY_cwp',
 'team2DUO_CARRY_ol',
 'team2DUO_CARRY_ow',


Next we need to re-order champions

In [123]:
champs = []
for num in range(0,10):
    columname= laneslist[num]
    champs = champs + list(matches_s6[columname].unique())

In [124]:
champs = list(set(champs))

In [125]:
len(champs)

133

In [126]:
champs2idx = {o:i for i,o in enumerate(champs)} # remove missing champions numbers and re-order

In [127]:
for num in range(0,10):
    columname= laneslist[num]
    matches_s6[columname]=matches_s6[columname].apply(lambda x: champs2idx[x])

/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  This is separate from the ipykernel package so we can avoid doing imports until


In [128]:
matches_s6.head()

,Unnamed: 0,patch_version,team1DUO_CARRY,team1DUO_CARRY_cl,team1DUO_CARRY_cpp,team1DUO_CARRY_cw,team1DUO_CARRY_cwp,team1DUO_CARRY_ol,team1DUO_CARRY_ow,team1DUO_CARRY_owp,...,team2TOP_cl,team2TOP_cpp,team2TOP_cw,team2TOP_cwp,team2TOP_ol,team2TOP_ow,team2TOP_owp,team2_win,team1,team2
927,1116,6.23.166.2676,71,2.0,0.004975,1.0,0.333333,287.0,316.0,0.524046,...,2.0,0.006369,1.0,0.333333,209.0,262.0,0.556263,True,0,1
928,1117,6.23.166.2676,115,2.0,0.002620,1.0,0.333333,548.0,597.0,0.521397,...,2.0,0.003030,1.0,0.333333,477.0,513.0,0.518182,True,0,1
929,1118,6.23.166.2676,98,2.0,0.029126,1.0,0.333333,32.0,71.0,0.689320,...,2.0,0.008065,1.0,0.333333,182.0,190.0,0.510753,False,1,0
930,1119,6.23.166.2676,71,2.0,0.004769,1.0,0.333333,260.0,369.0,0.586645,...,2.0,0.002611,1.0,0.333333,570.0,579.0,0.503916,False,1,0
931,1120,6.23.166.2676,115,2.0,0.004769,1.0,0.333333,260.0,369.0,0.586645,...,2.0,0.005068,1.0,0.333333,270.0,322.0,0.543919,False,1,0


In [129]:
n_hidden_factors = 16

In [130]:
msk1 = np.random.rand(len(matches_s6)) < 0.8 #training vs test set ratio This is the same structure I use in my anime_rec engine
trn1 = matches_s6[msk1]
val1 = matches_s6[~msk1]

In [131]:
def embedding_input_champion(name, n_in, n_out, reg):
    inp = Input(shape=(1,), dtype='int64', name=name)
    return inp, Embedding(n_in, n_out, input_length=1, W_regularizer=l2(reg))(inp)

In [132]:
champion_in1, c1 = embedding_input_champion('champion_id1_in', 134, n_hidden_factors,0)
champion_in2, c2 = embedding_input_champion('champion_id2_in', 134, n_hidden_factors, 0)
champion_in3, c3 = embedding_input_champion('champion_id3_in', 134, n_hidden_factors, 0)
champion_in4, c4 = embedding_input_champion('champion_id4_in', 134, n_hidden_factors, 0)
champion_in5, c5 = embedding_input_champion('champion_id5_in', 134, n_hidden_factors, 0)
champion_in6, c6 = embedding_input_champion('champion_id6_in', 134, n_hidden_factors, 0)
champion_in7, c7 = embedding_input_champion('champion_id7_in', 134, n_hidden_factors, 0)
champion_in8, c8 = embedding_input_champion('champion_id8_in', 134, n_hidden_factors, 0)
champion_in9, c9 = embedding_input_champion('champion_id9_in', 134, n_hidden_factors, 0)
champion_in10, c10 = embedding_input_champion('champion_id10_in', 134, n_hidden_factors, 0)

/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py:3: UserWarning: Update your `Embedding` call to the Keras 2 API: `Embedding(134, 16, embeddings_regularizer=<keras.reg..., input_length=1)`
  This is separate from the ipykernel package so we can avoid doing imports until


In [133]:
c11 = Input(shape=(70,))
x = merge([c1,c2,c3,c4,c5,c6,c7,c8,c9,c10], mode='concat', name='All_champion_Factors_on_one_layer')

x= Flatten()(x)
x=merge([x,c11],mode='concat')

x = Dense(140, activation='sigmoid',name='Random_HF_Interactions1')(x)
#x = Dropout(0.8,name='Prevent_overfit2')(x)
x = Dense(80, activation='relu',name='Random_HF_Interactions2')(x)
x = Dense(40, activation='sigmoid',name='Random_HF_Interactions3')(x)
x = Dense(1,activation='sigmoid',name='Final_Interactions')(x)
nn1 = Model([champion_in1,champion_in2,champion_in3,champion_in4,champion_in5,champion_in6,champion_in7,champion_in8,champion_in9,champion_in10,c11], x)
nn1.compile(Adam(0.001), loss='binary_crossentropy', metrics=['accuracy'])

/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py:2: UserWarning: The `merge` function is deprecated and will be removed after 08/2017. Use instead layers from `keras.layers.merge`, e.g. `add`, `concatenate`, etc.
  
/usr/local/lib/python3.5/dist-packages/keras/legacy/layers.py:458: UserWarning: The `Merge` layer is deprecated and will be removed after 08/2017. Use instead layers from `keras.layers.merge`, e.g. `add`, `concatenate`, etc.
  name=name)
/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py:5: UserWarning: The `merge` function is deprecated and will be removed after 08/2017. Use instead layers from `keras.layers.merge`, e.g. `add`, `concatenate`, etc.
  """


In [134]:
# now let's make an array of the 70 numeric features

In [135]:
featureslist =list(matches_s6.columns)

In [136]:
xfeatureslist =[
 'team1DUO_CARRY_cl',
 'team1DUO_CARRY_cpp',
 'team1DUO_CARRY_cw',
 'team1DUO_CARRY_cwp',
 'team1DUO_CARRY_ol',
 'team1DUO_CARRY_ow',
 'team1DUO_CARRY_owp',
 'team1DUO_SUPPORT_cl',
 'team1DUO_SUPPORT_cpp',
 'team1DUO_SUPPORT_cw',
 'team1DUO_SUPPORT_cwp',
 'team1DUO_SUPPORT_ol',
 'team1DUO_SUPPORT_ow',
 'team1DUO_SUPPORT_owp',
 'team1JUNGLE_cl',
 'team1JUNGLE_cpp',
 'team1JUNGLE_cw',
 'team1JUNGLE_cwp',
 'team1JUNGLE_ol',
 'team1JUNGLE_ow',
 'team1JUNGLE_owp',
 'team1MIDDLE_cl',
 'team1MIDDLE_cpp',
 'team1MIDDLE_cw',
 'team1MIDDLE_cwp',
 'team1MIDDLE_ol',
 'team1MIDDLE_ow',
 'team1MIDDLE_owp',
 'team1TOP_cl',
 'team1TOP_cpp',
 'team1TOP_cw',
 'team1TOP_cwp',
 'team1TOP_ol',
 'team1TOP_ow',
 'team1TOP_owp',
 'team2DUO_CARRY_cl',
 'team2DUO_CARRY_cpp',
 'team2DUO_CARRY_cw',
 'team2DUO_CARRY_cwp',
 'team2DUO_CARRY_ol',
 'team2DUO_CARRY_ow',
 'team2DUO_CARRY_owp',
 'team2DUO_SUPPORT_cl',
 'team2DUO_SUPPORT_cpp',
 'team2DUO_SUPPORT_cw',
 'team2DUO_SUPPORT_cwp',
 'team2DUO_SUPPORT_ol',
 'team2DUO_SUPPORT_ow',
 'team2DUO_SUPPORT_owp',
 'team2JUNGLE_cl',
 'team2JUNGLE_cpp',
 'team2JUNGLE_cw',
 'team2JUNGLE_cwp',
 'team2JUNGLE_ol',
 'team2JUNGLE_ow',
 'team2JUNGLE_owp',
 'team2MIDDLE_cl',
 'team2MIDDLE_cpp',
 'team2MIDDLE_cw',
 'team2MIDDLE_cwp',
 'team2MIDDLE_ol',
 'team2MIDDLE_ow',
 'team2MIDDLE_owp',
 'team2TOP_cl',
 'team2TOP_cpp',
 'team2TOP_cw',
 'team2TOP_cwp',
 'team2TOP_ol',
 'team2TOP_ow',
 'team2TOP_owp',
 ]

In [137]:
X=trn1[xfeatureslist].values

In [138]:
X_test= val1[xfeatureslist].values

In [139]:
nn1.fit([trn1.team1TOP,trn1.team2MIDDLE,trn1.team1JUNGLE,trn1.team1DUO_SUPPORT,trn1.team2DUO_CARRY,trn1.team2TOP,trn1.team2MIDDLE,trn1.team2JUNGLE,trn1.team2DUO_SUPPORT,trn1.team2DUO_CARRY,X],trn1.team1, batch_size=100, epochs=25, 
       validation_data=([val1.team1TOP,val1.team2MIDDLE,val1.team1JUNGLE,val1.team1DUO_SUPPORT,val1.team2DUO_CARRY,val1.team2TOP,val1.team2MIDDLE,val1.team2JUNGLE,val1.team2DUO_SUPPORT,val1.team2DUO_CARRY,X_test],val1.team1))

Train on 45731 samples, validate on 11387 samples
Epoch 1/25
45731/45731 [==============================] - 5s - loss: 0.6937 - acc: 0.5085 - val_loss: 0.6929 - val_acc: 0.5057
Epoch 2/25
45731/45731 [==============================] - 5s - loss: 0.6930 - acc: 0.5116 - val_loss: 0.6925 - val_acc: 0.5229
Epoch 3/25
45731/45731 [==============================] - 5s - loss: 0.6928 - acc: 0.5153 - val_loss: 0.6921 - val_acc: 0.5231
Epoch 4/25
45731/45731 [==============================] - 5s - loss: 0.6927 - acc: 0.5146 - val_loss: 0.6921 - val_acc: 0.5231
Epoch 5/25
45731/45731 [==============================] - 5s - loss: 0.6929 - acc: 0.5163 - val_loss: 0.6921 - val_acc: 0.5231
Epoch 6/25
45731/45731 [==============================] - 5s - loss: 0.6928 - acc: 0.5162 - val_loss: 0.6924 - val_acc: 0.5231
Epoch 7/25
45731/45731 [==============================] - 5s - loss: 0.6927 - acc: 0.5152 - val_loss: 0.6923 - val_acc: 0.5231
Epoch 8/25
45731/45731 [==============================] - 5s 

In [140]:
nn1.fit([trn1.team1TOP,trn1.team2MIDDLE,trn1.team1JUNGLE,trn1.team1DUO_SUPPORT,trn1.team2DUO_CARRY,trn1.team2TOP,trn1.team2MIDDLE,trn1.team2JUNGLE,trn1.team2DUO_SUPPORT,trn1.team2DUO_CARRY,X],trn1.team1, batch_size=1000, epochs=25, 
       validation_data=([val1.team1TOP,val1.team2MIDDLE,val1.team1JUNGLE,val1.team1DUO_SUPPORT,val1.team2DUO_CARRY,val1.team2TOP,val1.team2MIDDLE,val1.team2JUNGLE,val1.team2DUO_SUPPORT,val1.team2DUO_CARRY,X_test],val1.team1))

Train on 45731 samples, validate on 11387 samples
Epoch 1/25
45731/45731 [==============================] - 0s - loss: 0.6925 - acc: 0.5179 - val_loss: 0.6920 - val_acc: 0.5239
Epoch 2/25
45731/45731 [==============================] - 0s - loss: 0.6922 - acc: 0.5186 - val_loss: 0.6918 - val_acc: 0.5238
Epoch 3/25
45731/45731 [==============================] - 0s - loss: 0.6922 - acc: 0.5189 - val_loss: 0.6917 - val_acc: 0.5247
Epoch 4/25
45731/45731 [==============================] - 0s - loss: 0.6921 - acc: 0.5190 - val_loss: 0.6917 - val_acc: 0.5231
Epoch 5/25
45731/45731 [==============================] - 0s - loss: 0.6922 - acc: 0.5202 - val_loss: 0.6918 - val_acc: 0.5238
Epoch 6/25
45731/45731 [==============================] - 0s - loss: 0.6922 - acc: 0.5209 - val_loss: 0.6918 - val_acc: 0.5227
Epoch 7/25
45731/45731 [==============================] - 0s - loss: 0.6921 - acc: 0.5189 - val_loss: 0.6917 - val_acc: 0.5225
Epoch 8/25
45731/45731 [==============================] - 0s 

So we're not doing any better than if we just look at the champions picked. 
Given how much imputation had to occur, let's try with less factors. 


In [159]:
n_hidden_factors2 = 8

In [160]:
champion_in21, c21 = embedding_input_champion('champion_id1_in', 134, n_hidden_factors2,.001)
champion_in22, c22 = embedding_input_champion('champion_id2_in', 134, n_hidden_factors2, .001)
champion_in23, c23 = embedding_input_champion('champion_id3_in', 134, n_hidden_factors2, .001)
champion_in24, c24 = embedding_input_champion('champion_id4_in', 134, n_hidden_factors2, .001)
champion_in25, c25 = embedding_input_champion('champion_id5_in', 134, n_hidden_factors2, .001)
champion_in26, c26 = embedding_input_champion('champion_id6_in', 134, n_hidden_factors2, .001)
champion_in27, c27 = embedding_input_champion('champion_id7_in', 134, n_hidden_factors2, .001)
champion_in28, c28 = embedding_input_champion('champion_id8_in', 134, n_hidden_factors2, .001)
champion_in29, c29 = embedding_input_champion('champion_id9_in', 134, n_hidden_factors2, .001)
champion_in20, c20 = embedding_input_champion('champion_id10_in', 134, n_hidden_factors2, .001)

/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py:3: UserWarning: Update your `Embedding` call to the Keras 2 API: `Embedding(134, 8, embeddings_regularizer=<keras.reg..., input_length=1)`
  This is separate from the ipykernel package so we can avoid doing imports until


In [161]:
num_inputs = Input(shape=(30,))
x = merge([c21,c22,c23,c24,c25,c26,c27,c28,c29,c20], mode='concat', name='All_champion_Factors_on_one_layer')

x= Flatten()(x)
x=merge([x,num_inputs],mode='concat')

x = Dense(140, activation='sigmoid',name='Random_HF_Interactions1')(x)
x = Dropout(0.8,name='Prevent_overfit1')(x)
x = Dense(80, activation='sigmoid',name='Random_HF_Interactions2')(x)
x = Dense(40, activation='sigmoid',name='Random_HF_Interactions3')(x)
x = Dropout(0.8,name='Prevent_overfit2')(x)
x = Dense(1,activation='sigmoid',name='Final_Interactions')(x)
nn2 = Model([champion_in21,champion_in22,champion_in23,champion_in24,champion_in25,champion_in26,champion_in27,champion_in28,champion_in29,champion_in20,num_inputs], x)
nn2.compile(Adam(0.001), loss='binary_crossentropy', metrics=['accuracy'])

/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py:2: UserWarning: The `merge` function is deprecated and will be removed after 08/2017. Use instead layers from `keras.layers.merge`, e.g. `add`, `concatenate`, etc.
  
/usr/local/lib/python3.5/dist-packages/keras/legacy/layers.py:458: UserWarning: The `Merge` layer is deprecated and will be removed after 08/2017. Use instead layers from `keras.layers.merge`, e.g. `add`, `concatenate`, etc.
  name=name)
/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py:5: UserWarning: The `merge` function is deprecated and will be removed after 08/2017. Use instead layers from `keras.layers.merge`, e.g. `add`, `concatenate`, etc.
  """


In [144]:
matches_s6[xfeatureslist].describe() # After looking at this, I'm only going to look at percentages

,team1DUO_CARRY_cl,team1DUO_CARRY_cpp,team1DUO_CARRY_cw,team1DUO_CARRY_cwp,team1DUO_CARRY_ol,team1DUO_CARRY_ow,team1DUO_CARRY_owp,team1DUO_SUPPORT_cl,team1DUO_SUPPORT_cpp,team1DUO_SUPPORT_cw,...,team2MIDDLE_ol,team2MIDDLE_ow,team2MIDDLE_owp,team2TOP_cl,team2TOP_cpp,team2TOP_cw,team2TOP_cwp,team2TOP_ol,team2TOP_ow,team2TOP_owp
count,57118.000000,57118.000000,57118.000000,57118.000000,57118.000000,57118.000000,57118.000000,57118.000000,57118.000000,57118.000000,...,57118.000000,57118.000000,57118.000000,57118.000000,57118.000000,57118.000000,57118.000000,57118.000000,57118.000000,57118.000000
mean,3.708441,0.029264,3.198161,0.403698,214.738655,221.788517,0.507453,3.202743,0.027269,2.726285,...,222.171919,229.590924,0.507779,2.983338,0.025113,2.454847,0.402662,213.802083,220.923995,0.507837
std,2.348397,0.101500,2.912157,0.086346,143.329743,145.922401,0.036592,1.760490,0.105890,2.322340,...,155.877116,158.700815,0.036885,1.679124,0.095482,2.184544,0.086433,145.328958,147.791952,0.037468
min,1.940412,0.001009,1.000000,0.333333,0.000000,0.000000,0.000000,1.940412,0.001009,1.000000,...,0.000000,0.000000,0.000000,1.940412,0.001009,1.000000,0.333333,0.000000,0.000000,0.000000
25%,2.000000,0.006122,1.000000,0.333333,172.040667,178.941211,0.502041,2.000000,0.006536,1.000000,...,172.040667,178.941211,0.502242,2.000000,0.006048,1.000000,0.333333,169.000000,176.000000,0.502252
50%,2.000000,0.017066,1.000000,0.333333,172.040667,178.941211,0.509830,2.000000,0.017045,1.000000,...,172.040667,178.941211,0.509830,2.000000,0.015306,1.000000,0.333333,172.040667,178.941211,0.509830
75%,6.453734,0.037201,6.099135,0.505718,242.000000,250.000000,0.509830,4.194892,0.027768,4.591261,...,250.000000,260.000000,0.510309,3.562674,0.024852,3.591020,0.507499,241.000000,252.000000,0.510728
max,8.075272,3.000000,8.661634,0.549259,1466.000000,1506.000000,1.000000,9.949848,3.000000,9.610718,...,1466.000000,1506.000000,1.000000,9.949848,3.000000,9.610718,0.553484,1466.000000,1506.000000,1.000000


In [145]:
x2featureslist =[
 'team1DUO_CARRY_cwp',
 'team1DUO_CARRY_owp',
 'team1DUO_SUPPORT_cwp',
'team1DUO_SUPPORT_owp',
  'team1JUNGLE_cwp',
  'team1JUNGLE_owp',
 'team1MIDDLE_cwp',
'team1MIDDLE_owp',
 'team1TOP_cwp',
'team1TOP_owp',
 'team2DUO_CARRY_cwp',
  'team2DUO_CARRY_owp',
 'team2DUO_SUPPORT_cwp',
 'team2DUO_SUPPORT_owp',
 'team2JUNGLE_cwp',
 'team2JUNGLE_owp',
  'team2MIDDLE_cwp',
'team2MIDDLE_owp',
 'team2TOP_cwp',
 'team2TOP_owp',
'team1MIDDLE_cpp',
    'team1TOP_cpp',
    'team1JUNGLE_cpp',
    'team1DUO_CARRY_cpp',
    'team1DUO_SUPPORT_cpp',
    'team2MIDDLE_cpp',
    'team2TOP_cpp',
    'team2JUNGLE_cpp',
    'team2DUO_CARRY_cpp',
    'team2DUO_SUPPORT_cpp',
 ]

In [146]:
len(x2featureslist)

30

Now we have decided what 3 variables per player we are lookin at (s7 win rate on champion, s7 win rate, playrate of champion in ranked)

In [147]:
X2=trn1[x2featureslist].values

In [148]:
X2_test= val1[x2featureslist].values

In [149]:
nn2.fit([trn1.team1TOP,trn1.team2MIDDLE,trn1.team1JUNGLE,trn1.team1DUO_SUPPORT,trn1.team2DUO_CARRY,trn1.team2TOP,trn1.team2MIDDLE,trn1.team2JUNGLE,trn1.team2DUO_SUPPORT,trn1.team2DUO_CARRY,X2],trn1.team1, batch_size=100, epochs=25, 
       validation_data=([val1.team1TOP,val1.team2MIDDLE,val1.team1JUNGLE,val1.team1DUO_SUPPORT,val1.team2DUO_CARRY,val1.team2TOP,val1.team2MIDDLE,val1.team2JUNGLE,val1.team2DUO_SUPPORT,val1.team2DUO_CARRY,X2_test],val1.team1))

Train on 45731 samples, validate on 11387 samples
Epoch 1/25
45731/45731 [==============================] - 6s - loss: 0.7129 - acc: 0.5073 - val_loss: 0.6921 - val_acc: 0.5231
Epoch 2/25
45731/45731 [==============================] - 5s - loss: 0.6935 - acc: 0.5085 - val_loss: 0.6922 - val_acc: 0.5231
Epoch 3/25
45731/45731 [==============================] - 5s - loss: 0.6928 - acc: 0.5167 - val_loss: 0.6922 - val_acc: 0.5231
Epoch 4/25
45731/45731 [==============================] - 5s - loss: 0.6926 - acc: 0.5169 - val_loss: 0.6921 - val_acc: 0.5231
Epoch 5/25
45731/45731 [==============================] - 5s - loss: 0.6926 - acc: 0.5175 - val_loss: 0.6921 - val_acc: 0.5231
Epoch 6/25
45731/45731 [==============================] - 5s - loss: 0.6926 - acc: 0.5168 - val_loss: 0.6922 - val_acc: 0.5231
Epoch 7/25
45731/45731 [==============================] - 5s - loss: 0.6927 - acc: 0.5175 - val_loss: 0.6922 - val_acc: 0.5231
Epoch 8/25
45731/45731 [==============================] - 5s 

In [163]:
nn2.fit([trn1.team1TOP,trn1.team2MIDDLE,trn1.team1JUNGLE,trn1.team1DUO_SUPPORT,trn1.team2DUO_CARRY,trn1.team2TOP,trn1.team2MIDDLE,trn1.team2JUNGLE,trn1.team2DUO_SUPPORT,trn1.team2DUO_CARRY,X2],trn1.team1, batch_size=1000, epochs=25, 
       validation_data=([val1.team1TOP,val1.team2MIDDLE,val1.team1JUNGLE,val1.team1DUO_SUPPORT,val1.team2DUO_CARRY,val1.team2TOP,val1.team2MIDDLE,val1.team2JUNGLE,val1.team2DUO_SUPPORT,val1.team2DUO_CARRY,X2_test],val1.team1))

Train on 45731 samples, validate on 11387 samples
Epoch 1/25
45731/45731 [==============================] - 0s - loss: 0.6932 - acc: 0.5151 - val_loss: 0.6922 - val_acc: 0.5231
Epoch 2/25
45731/45731 [==============================] - 0s - loss: 0.6934 - acc: 0.5062 - val_loss: 0.6923 - val_acc: 0.5231
Epoch 3/25
45731/45731 [==============================] - 0s - loss: 0.6928 - acc: 0.5167 - val_loss: 0.6922 - val_acc: 0.5231
Epoch 4/25
45731/45731 [==============================] - 0s - loss: 0.6928 - acc: 0.5162 - val_loss: 0.6922 - val_acc: 0.5231
Epoch 5/25
45731/45731 [==============================] - 0s - loss: 0.6928 - acc: 0.5163 - val_loss: 0.6922 - val_acc: 0.5231
Epoch 6/25
45731/45731 [==============================] - 0s - loss: 0.6927 - acc: 0.5155 - val_loss: 0.6922 - val_acc: 0.5231
Epoch 7/25
45731/45731 [==============================] - 0s - loss: 0.6927 - acc: 0.5143 - val_loss: 0.6922 - val_acc: 0.5231
Epoch 8/25
45731/45731 [==============================] - 0s 

In [162]:
nn2.fit([trn1.team1TOP,trn1.team2MIDDLE,trn1.team1JUNGLE,trn1.team1DUO_SUPPORT,trn1.team2DUO_CARRY,trn1.team2TOP,trn1.team2MIDDLE,trn1.team2JUNGLE,trn1.team2DUO_SUPPORT,trn1.team2DUO_CARRY,X2],trn1.team1, batch_size=4000, epochs=25, 
       validation_data=([val1.team1TOP,val1.team2MIDDLE,val1.team1JUNGLE,val1.team1DUO_SUPPORT,val1.team2DUO_CARRY,val1.team2TOP,val1.team2MIDDLE,val1.team2JUNGLE,val1.team2DUO_SUPPORT,val1.team2DUO_CARRY,X2_test],val1.team1))

Train on 45731 samples, validate on 11387 samples
Epoch 1/25
45731/45731 [==============================] - 1s - loss: 0.8943 - acc: 0.5056 - val_loss: 0.6968 - val_acc: 0.5231
Epoch 2/25
45731/45731 [==============================] - 0s - loss: 0.7924 - acc: 0.5046 - val_loss: 0.6939 - val_acc: 0.5231
Epoch 3/25
45731/45731 [==============================] - 0s - loss: 0.7429 - acc: 0.5013 - val_loss: 0.6926 - val_acc: 0.5231
Epoch 4/25
45731/45731 [==============================] - 0s - loss: 0.7163 - acc: 0.5064 - val_loss: 0.6922 - val_acc: 0.5231
Epoch 5/25
45731/45731 [==============================] - 0s - loss: 0.7066 - acc: 0.4999 - val_loss: 0.6922 - val_acc: 0.5231
Epoch 6/25
45731/45731 [==============================] - 0s - loss: 0.7006 - acc: 0.5047 - val_loss: 0.6922 - val_acc: 0.5231
Epoch 7/25
45731/45731 [==============================] - 0s - loss: 0.6988 - acc: 0.5046 - val_loss: 0.6922 - val_acc: 0.5231
Epoch 8/25
45731/45731 [==============================] - 0s 

Conclusion: My imputation of data is so severely noisy that is is reducing the model's accuracy.  I should redo this project from the data collection stage. 

In [165]:
!pip install xgboost

    100% |################################| 1.2MB 757kB/s ta 0:00:01
  Running setup.py bdist_wheel for xgboost ... done
  Stored in directory: /root/.cache/pip/wheels/5e/c1/d6/522af54e5cc001fad4dd855117f8bf61b11d56443e06672e26
Successfully built xgboost


In [166]:
from xgboost import XGBClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

/usr/local/lib/python3.5/dist-packages/sklearn/cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


In [167]:
laneslist = [
    'team1TOP',
    'team1MIDDLE',
    'team1JUNGLE',
    'team1DUO_CARRY',
    'team1DUO_SUPPORT',
    'team2TOP',
    'team2MIDDLE',
    'team2JUNGLE',
    'team2DUO_CARRY',
    'team2DUO_SUPPORT'
    
    
]

In [177]:
xgbfeatureslist =[
 'team1DUO_CARRY_cwp',
 'team1DUO_CARRY_owp',
 'team1DUO_SUPPORT_cwp',
'team1DUO_SUPPORT_owp',
  'team1JUNGLE_cwp',
  'team1JUNGLE_owp',
 'team1MIDDLE_cwp',
'team1MIDDLE_owp',
 'team1TOP_cwp',
'team1TOP_owp',
 'team2DUO_CARRY_cwp',
  'team2DUO_CARRY_owp',
 'team2DUO_SUPPORT_cwp',
 'team2DUO_SUPPORT_owp',
 'team2JUNGLE_cwp',
 'team2JUNGLE_owp',
  'team2MIDDLE_cwp',
'team2MIDDLE_owp',
 'team2TOP_cwp',
 'team2TOP_owp',
'team1MIDDLE_cpp',
    'team1TOP_cpp',
    'team1JUNGLE_cpp',
    'team1DUO_CARRY_cpp',
    'team1DUO_SUPPORT_cpp',
    'team2MIDDLE_cpp',
    'team2TOP_cpp',
    'team2JUNGLE_cpp',
    'team2DUO_CARRY_cpp',
    'team2DUO_SUPPORT_cpp',
        'team1TOP',
    'team1MIDDLE',
    'team1JUNGLE',
    'team1DUO_CARRY',
    'team1DUO_SUPPORT',
    'team2TOP',
    'team2MIDDLE',
    'team2JUNGLE',
    'team2DUO_CARRY',
    'team2DUO_SUPPORT',
    'team1'
 ]

In [178]:
xgb_data= matches_s6[xgbfeatureslist]

In [181]:
xgb_data.head()

,team1DUO_CARRY_cwp,team1DUO_CARRY_owp,team1DUO_SUPPORT_cwp,team1DUO_SUPPORT_owp,team1JUNGLE_cwp,team1JUNGLE_owp,team1MIDDLE_cwp,team1MIDDLE_owp,team1TOP_cwp,team1TOP_owp,...,team1MIDDLE,team1JUNGLE,team1DUO_CARRY,team1DUO_SUPPORT,team2TOP,team2MIDDLE,team2JUNGLE,team2DUO_CARRY,team2DUO_SUPPORT,team1
927,0.333333,0.524046,0.333333,0.511081,0.333333,0.549020,0.333333,0.562278,0.333333,0.491061,...,50,87,71,58,12,75,34,115,97,0
928,0.333333,0.521397,0.333333,0.579235,0.333333,0.562278,0.333333,0.516129,0.333333,0.509709,...,58,59,115,42,102,12,129,47,125,0
929,0.333333,0.689320,0.333333,0.498039,0.333333,0.549020,0.333333,0.518182,0.333333,0.515860,...,93,87,98,125,68,41,123,28,114,1
930,0.333333,0.586645,0.333333,0.568966,0.333333,0.595797,0.333333,0.530822,0.333333,0.520000,...,111,110,71,16,128,3,1,28,114,1
931,0.333333,0.586645,0.333333,0.536748,0.333333,0.595797,0.333333,0.562278,0.333333,0.536273,...,50,110,115,113,102,122,59,47,42,1


In [184]:
xgb_data_dummied= pd.get_dummies(xgb_data, columns=laneslist)

In [186]:
X= xgb_data_dummied.drop('team1',axis=1)

In [187]:
y=xgb_data_dummied['team1']

So, Now we've converted all of our champions into dummy variables, and we can just feed the whole thing to xgboost. 

In [188]:
test_size = 0.2
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=test_size)

In [189]:
# fit model training data
model = XGBClassifier()
model.fit(X_train, y_train)

XGBClassifier(base_score=0.5, colsample_bylevel=1, colsample_bytree=1,
       gamma=0, learning_rate=0.1, max_delta_step=0, max_depth=3,
       min_child_weight=1, missing=None, n_estimators=100, nthread=-1,
       objective='binary:logistic', reg_alpha=0, reg_lambda=1,
       scale_pos_weight=1, seed=0, silent=True, subsample=1)

In [191]:

# make predictions for test data
y_pred = model.predict(X_test)
predictions = [round(value) for value in y_pred]

In [192]:

# evaluate predictions
accuracy = accuracy_score(y_test, predictions)
print("Accuracy: %.2f%%" % (accuracy * 100.0))

Accuracy: 54.39%


So we're getting the same results with xgboost we're getting with a neural net. Let's see what happens  with more computation before we try with fewer variables. 

In [216]:

# fit model no training data
model2 = XGBClassifier(max_depth=7,n_estimators=500)
model2.fit(X_train, y_train)

XGBClassifier(base_score=0.5, colsample_bylevel=1, colsample_bytree=1,
       gamma=0, learning_rate=0.1, max_delta_step=0, max_depth=7,
       min_child_weight=1, missing=None, n_estimators=500, nthread=-1,
       objective='binary:logistic', reg_alpha=0, reg_lambda=1,
       scale_pos_weight=1, seed=0, silent=True, subsample=1)

In [215]:
y_pred2 = model2.predict(X_test)
predictions = [round(value) for value in y_pred2]
accuracy = accuracy_score(y_test, predictions)
print("Accuracy: %.2f%%" % (accuracy * 100.0))

Accuracy: 52.70%


In [199]:
xbg2laneslist = [
    'team1',
    'team1TOP',
    'team1MIDDLE',
    'team1JUNGLE',
    'team1DUO_CARRY',
    'team1DUO_SUPPORT',
    'team2TOP',
    'team2MIDDLE',
    'team2JUNGLE',
    'team2DUO_CARRY',
    'team2DUO_SUPPORT'
    
    
]

In [200]:
xgb_data2= matches_s6[xbg2laneslist]

In [201]:
xgb_data_dummied2= pd.get_dummies(xgb_data2, columns=laneslist)

In [204]:
X2= xgb_data_dummied.drop('team1',axis=1)
y2=xgb_data_dummied['team1']

In [205]:
test_size = 0.2
X2_train, X2_test, y2_train, y2_test = train_test_split(X2, y2, test_size=test_size)

In [243]:

# fit model no training data
model3 = XGBClassifier(max_depth=4,n_estimators=200)
model3.fit(X2_train, y2_train)

XGBClassifier(base_score=0.5, colsample_bylevel=1, colsample_bytree=1,
       gamma=0, learning_rate=0.1, max_delta_step=0, max_depth=4,
       min_child_weight=1, missing=None, n_estimators=200, nthread=-1,
       objective='binary:logistic', reg_alpha=0, reg_lambda=1,
       scale_pos_weight=1, seed=0, silent=True, subsample=1)

In [244]:
y_pred3 = model3.predict(X2_test)
predictions3 = [round(value) for value in y_pred3]

In [245]:
accuracy = accuracy_score(y2_test, predictions3)
print("Accuracy: %.2f%%" % (accuracy * 100.0))

Accuracy: 54.32%


Getting 54 percent success with XGBOOSt, 55 with a factorization machine neural network with keras is I suspect the limits with my current dataset. 

How to improve data (and require less imputation) for future versions of this project:

1. Scrape more matches. 
Lack of matches given the relative sparsity of the dataset (133 active champions in 10 roles means many combinations are rare)

2. Redesign the scraper to ignore summary statistics. 
All of the summary statistics appear to make models worse. 
Doing summary statistics off of data gathered would produce better outcomes. 

3. Redesign the scrape method to prioritize fewer different summoners:
Psuedo code for a better scrape method would be:
scrape 200 top players. Get all matches they have played. 
Get all players who have played with the top 200 in all of teir matches . 
Stop getting new scrape players at that point. 
Restart at beginning of loop. 

4. Benefits of part 3 pseudo-code:
Having fewer players to scrape means that it's possible to generate our own summary statistics:
For each champion in each patch, generate overall w/l and games played. 
For each player, generate overall win/loss, win/loss per champion, and champion play percentage. 
Because none of these are imputed, there should be less duplicate values as in our currently imputed dataset. 


## Post Script: XGBoost imputation:
We can go back to the dataprep notebook, and re-process data without imputation. 
Then we'll run xgboost with that. 

In [246]:
nonimputed= pd.read_csv('matches_no_impute.csv')

In [247]:
nonimputed.head()

,Unnamed: 0,MatchID,matchid,matchtime,patch_version,player10champid,player10lane,player10role,player10summonerid,player10summonername,...,team2SOLO_owp,team2TOP,team2TOP_cl,team2TOP_cpp,team2TOP_cw,team2TOP_cwp,team2TOP_ol,team2TOP_ow,team2TOP_owp,team2_win
0,0,2.078664e+09,2078663920,2016-01-25 05:06:24.884000,6.1.0.484,4.0,MIDDLE,SOLO,21498922.0,TR Linsanity,...,NaN,114.0,NaN,NaN,NaN,NaN,181.0,178.0,0.495822,False
1,1,2.078608e+09,2078608407,2016-01-25 04:25:30.551000,6.1.0.484,41.0,TOP,SOLO,28039649.0,Chunkylol,...,NaN,41.0,2.0,0.004990,3.0,0.600000,483.0,519.0,0.517964,True
2,2,2.078592e+09,2078592445,2016-01-25 03:52:01.302000,6.1.0.484,40.0,BOTTOM,DUO_SUPPORT,19806051.0,TR Strife,...,NaN,24.0,NaN,NaN,NaN,NaN,716.0,751.0,0.511929,True
3,3,2.078495e+09,2078494759,2016-01-25 03:13:55.927000,6.1.0.484,40.0,BOTTOM,DUO_SUPPORT,465551.0,lIlIlIIIIlIIIlII,...,NaN,27.0,29.0,0.827586,43.0,0.597222,39.0,48.0,0.551724,True
4,4,2.078438e+09,2078437727,2016-01-25 02:41:39.662000,6.1.0.484,42.0,MIDDLE,SOLO,22005573.0,caaaaaaaaaaaaake,...,NaN,114.0,3.0,0.004772,4.0,0.571429,716.0,751.0,0.511929,True


In [248]:
nonimputed['team1']=nonimputed['team1_win'].astype('int')
nonimputed['team2']=nonimputed['team2_win'].astype('int')
nonimputed.head()

,Unnamed: 0,MatchID,matchid,matchtime,patch_version,player10champid,player10lane,player10role,player10summonerid,player10summonername,...,team2TOP_cl,team2TOP_cpp,team2TOP_cw,team2TOP_cwp,team2TOP_ol,team2TOP_ow,team2TOP_owp,team2_win,team1,team2
0,0,2.078664e+09,2078663920,2016-01-25 05:06:24.884000,6.1.0.484,4.0,MIDDLE,SOLO,21498922.0,TR Linsanity,...,NaN,NaN,NaN,NaN,181.0,178.0,0.495822,False,1,0
1,1,2.078608e+09,2078608407,2016-01-25 04:25:30.551000,6.1.0.484,41.0,TOP,SOLO,28039649.0,Chunkylol,...,2.0,0.004990,3.0,0.600000,483.0,519.0,0.517964,True,0,1
2,2,2.078592e+09,2078592445,2016-01-25 03:52:01.302000,6.1.0.484,40.0,BOTTOM,DUO_SUPPORT,19806051.0,TR Strife,...,NaN,NaN,NaN,NaN,716.0,751.0,0.511929,True,0,1
3,3,2.078495e+09,2078494759,2016-01-25 03:13:55.927000,6.1.0.484,40.0,BOTTOM,DUO_SUPPORT,465551.0,lIlIlIIIIlIIIlII,...,29.0,0.827586,43.0,0.597222,39.0,48.0,0.551724,True,0,1
4,4,2.078438e+09,2078437727,2016-01-25 02:41:39.662000,6.1.0.484,42.0,MIDDLE,SOLO,22005573.0,caaaaaaaaaaaaake,...,3.0,0.004772,4.0,0.571429,716.0,751.0,0.511929,True,0,1


In [252]:
list(nonimputed.columns)

['Unnamed: 0',
 'MatchID',
 'matchid',
 'matchtime',
 'patch_version',
 'player10champid',
 'player10lane',
 'player10role',
 'player10summonerid',
 'player10summonername',
 'player1champid',
 'player1lane',
 'player1role',
 'player1summonerid',
 'player1summonername',
 'player2champid',
 'player2lane',
 'player2role',
 'player2summonerid',
 'player2summonername',
 'player3champid',
 'player3lane',
 'player3role',
 'player3summonerid',
 'player3summonername',
 'player4champid',
 'player4lane',
 'player4role',
 'player4summonerid',
 'player4summonername',
 'player5champid',
 'player5lane',
 'player5role',
 'player5summonerid',
 'player5summonername',
 'player6champid',
 'player6lane',
 'player6role',
 'player6summonerid',
 'player6summonername',
 'player7champid',
 'player7lane',
 'player7role',
 'player7summonerid',
 'player7summonername',
 'player8champid',
 'player8lane',
 'player8role',
 'player8summonerid',
 'player8summonername',
 'player9champid',
 'player9lane',
 'player9role',


In [253]:
xgb_nonimpute_data= nonimputed[xgbfeatureslist]

In [254]:
xgb_nonimpute_data.head()

,team1DUO_CARRY_cwp,team1DUO_CARRY_owp,team1DUO_SUPPORT_cwp,team1DUO_SUPPORT_owp,team1JUNGLE_cwp,team1JUNGLE_owp,team1MIDDLE_cwp,team1MIDDLE_owp,team1TOP_cwp,team1TOP_owp,...,team1MIDDLE,team1JUNGLE,team1DUO_CARRY,team1DUO_SUPPORT,team2TOP,team2MIDDLE,team2JUNGLE,team2DUO_CARRY,team2DUO_SUPPORT,team1
0,0.500000,0.578667,0.503400,0.509830,0.272727,0.514720,0.500000,0.512658,0.769231,0.545894,...,13.0,245.0,42.0,412.0,114.0,4.0,64.0,236.0,40.0,1
1,0.485753,0.509830,0.499603,0.509830,0.500000,0.495822,0.587719,0.555556,NaN,0.511929,...,74.0,36.0,429.0,111.0,41.0,134.0,421.0,42.0,25.0,0
2,NaN,0.666667,0.633333,0.520101,0.333333,0.510753,0.000000,0.517964,0.485425,0.509830,...,13.0,203.0,17.0,25.0,24.0,117.0,76.0,236.0,40.0,0
3,0.505718,0.509830,0.555556,0.504837,NaN,0.550403,0.000000,0.521531,0.000000,0.513839,...,81.0,203.0,67.0,432.0,27.0,4.0,NaN,236.0,40.0,0
4,0.593750,0.558824,NaN,0.562278,0.666667,0.510753,0.400000,0.532407,0.485425,0.509830,...,7.0,76.0,236.0,12.0,114.0,42.0,203.0,81.0,40.0,0


In [255]:
xgb_nonimpute_dummied= pd.get_dummies(xgb_nonimpute_data, columns=laneslist)

In [256]:
X4= xgb_data_dummied.drop('team1',axis=1)
y4=xgb_data_dummied['team1']

In [257]:
test_size = 0.2
X4_train, X4_test, y4_train, y4_test = train_test_split(X4, y4, test_size=test_size)

In [258]:
model4 = XGBClassifier()
model4.fit(X4_train, y4_train)

XGBClassifier(base_score=0.5, colsample_bylevel=1, colsample_bytree=1,
       gamma=0, learning_rate=0.1, max_delta_step=0, max_depth=3,
       min_child_weight=1, missing=None, n_estimators=100, nthread=-1,
       objective='binary:logistic', reg_alpha=0, reg_lambda=1,
       scale_pos_weight=1, seed=0, silent=True, subsample=1)

In [259]:
y_pred4 = model4.predict(X4_test)
predictions4 = [round(value) for value in y_pred4]

In [260]:
accuracy = accuracy_score(y4_test, predictions4)
print("Accuracy: %.2f%%" % (accuracy * 100.0))

Accuracy: 53.36%


In [261]:
from sklearn.model_selection import  GridSearchCV

In [263]:
params= {'max_depth':[3,4,5,6,7,8], 'n_estimators':[100,150,200], 'subsample':[.5,.8,1]}
gs1 = GridSearchCV(model4, param_grid=params)

In [264]:
gs1.fit(X4_train,y4_train)

GridSearchCV(cv=None, error_score='raise',
       estimator=XGBClassifier(base_score=0.5, colsample_bylevel=1, colsample_bytree=1,
       gamma=0, learning_rate=0.1, max_delta_step=0, max_depth=3,
       min_child_weight=1, missing=None, n_estimators=100, nthread=-1,
       objective='binary:logistic', reg_alpha=0, reg_lambda=1,
       scale_pos_weight=1, seed=0, silent=True, subsample=1),
       fit_params=None, iid=True, n_jobs=1,
       param_grid={'subsample': [0.5, 0.8, 1], 'n_estimators': [100, 150, 200], 'max_depth': [3, 4, 5, 6, 7, 8]},
       pre_dispatch='2*n_jobs', refit=True, return_train_score=True,
       scoring=None, verbose=0)

In [265]:
gs1.best_params_

{'max_depth': 4, 'n_estimators': 150, 'subsample': 0.8}

In [266]:
gs1.best_score_

0.54652689631023765